#  Unit 2.4a Using Programs with Data, SQLAlchemy
> Using Programs with Data is focused on SQL and database actions.  Part A focuses on SQLAlchemy and an OOP programming style,
- toc: true
- image: /images/python.png
- categories: []
- type: ap
- week: 26

# Database and SQLAlchemy
> In this blog we will explore using programs with data, focused on Databases.  We will use SQLite Database to learn more about using Programs with Data.

**Focus on objects and objects as being data**

- College Board talks about ideas like 
    - Program Usage. "iterative and interactive way when processing information"
    - Managing Data.  "classifying data are part of the process in using programs", "data files in a Table"
    - Insight "insight and knowledge can be obtained from ...  digitally represented information"
    - Filter systems. 'tools for finding information and recognizing patterns"
    - Application. "the preserve has two databases", "an employee wants to count the number of book"

- PBL, Databases, Iterative/OOP
    - Iterative. Refers to a sequence of instructions or code being repeated until a specific end result is achieved
    - OOP. A computer programming model that organizes software design around data, or objects, rather than functions and logic
    - SQL. Structured Query Language, abbreviated as SQL, is a language used in programming, managing, and structuring data 


## Imports and Flask Objects
> Defines and key object creations

- Comment on where you have observed these working?
1. Flask app object
2. SQLAlchemy object


In [83]:
# debug : the app configs
# see the info on the configs because the data is translated, app is assigned the attributes... called setters
"""
These imports define the key objects
"""

from flask import Flask
from flask_sqlalchemy import SQLAlchemy

"""
These object and definitions are used throughout the Jupyter Notebook.
"""
# app and db were used throughout the year
# Setup of key Flask object (app)
app = Flask(__name__)
# Setup SQLAlchemy object and properties for the database (db)
database = 'sqlite:///sqlite.db'  # path and filename of database
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
app.config['SQLALCHEMY_DATABASE_URI'] = database
app.config['SECRET_KEY'] = 'SECRET_KEY'
# made a db and got it started
db = SQLAlchemy() 

# break point mean wan to examine the things above which have already happened
# This belongs in place where it runs once per project
db.init_app(app)


## Model Definition
> Define columns, initialization, and CRUD methods for users table in sqlite.db

- Comment on these items in the class
- class User purpose
- db.Model inheritance
- _init_ method
- @property, @<column>.setter
- additional methods

In [84]:
""" database dependencies to support sqlite examples """
import datetime
from datetime import datetime
import json

from sqlalchemy.exc import IntegrityError
from werkzeug.security import generate_password_hash, check_password_hash

class UCar(db.Model):
    __tablename__ = 'CarUser'  

    id = db.Column(db.Integer, primary_key=True) 
    _firstname = db.Column(db.String(255), unique=False, nullable=False)
    _lastname = db.Column(db.String(255), unique=False, nullable=False)
    _type = db.Column(db.String(255), unique=False, nullable=False)
    _license = db.Column(db.String(255), unique=True, nullable=False)
    _phone = db.Column(db.String(255), unique=False, nullable=False)
    _cpd = db.Column(db.Date)

    # constructor of a User object, initializes the instance variables within object (self)
    def __init__(self, firstname, lastname, type, license, phone="123qwerty", cpd=datetime.today()):
        self._firstname = firstname
        self._lastname = lastname
        self._type = type    # variables with self prefix become part of the object, 
        self._license = license
        self.set_(phone)
        if isinstance(cpd, str):  # not a date type     
            cpd = date=datetime.today()
        
    # name of person
    @property
    def firstname(self):
        return self._firstname

    @firstname.setter
    def firstname(self, firstname):
        self._firstname = firstname
    
    @property
    def lastname(self):
        return self._lastname

    @lastname.setter
    def lastname(self, lastname):
        self._lastname = lastname
    
    #type of car
    @property
    def type(self):
        return self._type

    @type.setter
    def name(self, type):
        self._type = type
    
    # license of car
    @property
    def license(self):
        return self._license
    
    # a setter function (update)
    @license.setter
    def license(self, license):
        self._license = license
        
    # return boolean
    def is_license(self, license):
        return self._license == license
    
    # phone number
    @property
    def phone(self):
        return self._phone[0:10] + "..." 
    # show only first 3

    # update password
    def set_phone(self, phone):
        """Create a hashed password."""
        self._phone = generate_password_hash(phone, method='sha256')
    
    # output content using str(object) in human readable form, uses getter
    # output content using json dumps, this is ready for API response
    def __str__(self):
        return json.dumps(self.read())

    # CRUD create/add a new record to the table
    # returns self or None on error
    def create(self):
        try:
            # creates a person object from User(db.Model) class, passes initializers
            db.session.add(self)  # add prepares to persist person object to Users table
            db.session.commit()  # SqlAlchemy "unit of work pattern" requires a manual commit
            return self
        except IntegrityError:
            db.session.remove()
            return None

# in class not only is there attributes there are also methods
    # CRUD read converts self to dictionary
    # returns dictionary
    def read(self):
        return {
            "id": self.id,
            "first name": self.firstname,
            "last name": self.lastname,
            "phone number": self.phone,
            "type of car": self.type,
            "license": self.license
        }

    # CRUD update: updates user name, password, phone
    # returns self
    def update(self, type="", lastname="", firstname="", license="", phone=""):
        """only updates values with length"""
        if len(lastname) > 0:
            self.lastname = lastname
        if len(firstname) > 0:
            self.firstname = firstname
        if len(license) > 0:
            self.license = license
        #if len(type) > 0:
            #self.type = type
        if len(phone) > 0:
            self.set_phone(phone)
        db.session.commit()
        return self

    # CRUD delete: remove self
    # None
    def delete(self):
        db.session.delete(self)
        db.session.commit()
        return None

## Initial Data
> Uses SQLALchemy db.create_all() to initialize rows into sqlite.db

- Comment on how these work?
1. Create All Tables from db Object
2. User Object Constructors
3. Try / Except 


In [86]:
# Builds working data for testing
def initCarUser():
    with app.app_context():
        """Create database and tables"""
        db.create_all()
        """Tester data for table"""
        C1 = UCar(firstname='Bob', lastname='Edison', phone='1232133724', type='minivan', license='123456')
        C2 = UCar(firstname='Adam', lastname='Adams', phone='1232342345', type='subaru', license='039278')
        C3 = UCar(firstname='Emily', lastname='Russo', phone='8582343456', type='cooper', license='394028')
        C4 = UCar(firstname='Jillian', lastname='Esa', phone='7603452904', type='van', license='473829')
        C5 = UCar(firstname='Billy', lastname='Richards', phone='8583742049', type='sports', license='583920')
        C6 = UCar(firstname='John', lastname='Adams', phone='7602947389', type='minivan', license='738290')
        CUser = [C1, C2, C3, C4, C5, C6]
        for C in CUser:
            try:
                object = C.create()
                print(f"New Data For {object.firstname}")
            except IntegrityError:  # error raised if object nit created
                '''fails with bad or duplicate data'''
                print(f"Records exist duplicate {C.Car}.")
                
initCarUser()

AttributeError: 'UCar' object has no attribute 'set_'

## Check for given Credentials in users table in sqlite.db
> Use of ORM Query object and custom methods to identify user to credentials uid and password

- Comment on purpose of following
1. User.query.filter_by
2. user.password

In [27]:
# SQLAlchemy extracts single user from database matching User ID
def find_by_firstname(firstname):
    with app.app_context():
        car = Car.query.filter_by(_firstname=firstname).first()
    return car # returns user object

# Check credentials by finding user and verify password
def check_credentials(firstname, phone):
    # query email and return user record
    car = find_by_firstname(firstname)
    if car == None:
        return False
    if (car.is_phone(phone)):
        return True
    return False
        
check_credentials("Billy", "8583742049")

False

## Create a new User in table in Sqlite.db
> Uses SQLALchemy and custom user.create() method to add row.

- Comment on purpose of following
1. user.find_by_uid() and try/except
2. user = User(...)
3. user.dob and try/except
4. user.create() and try/except

In [26]:
# Inputs, Try/Except, and SQLAlchemy work together to build a valid database object
def create():
    
    # request value that ensure creating valid object
    firstname = input("Enter your first name:")
    lastname = input("Enter your last name:")
    type = input("Enter the type of your car")
    phone = input("Enter your phone number")
    license = input("Enter your car license number")
    
    # Initialize User object before date
    Car = car(license==license, 
                firstname=firstname, 
                lastname=lastname,
                phone=phone,
                type=type
                )

    # write object to database
    with app.app_context():
        try:
            object = car.create()
            print("Created\n", object.read())
        except:  # error raised if object not created
            print("Unknown error license {firstname}")
        
create()

NameError: name 'car' is not defined

## Reading users table in sqlite.db
> Uses SQLALchemy query.all method to read data

- Comment on purpose of following
1. User.query.all
2. json_ready assignment

In [23]:

# SQLAlchemy extracts all users from database, turns each user into JSON
def read():
    with app.app_context():
        table = Car.query.all()
    json_ready = [car.read() for car in table] # each user adds user.read() to list
    return json_ready

read()

[]

# Hacks
- Add this Blog to you own Blogging site.  In the Blog add notes and observations on each code cell.
- Add Update functionality to this blog.
- Add Delete functionality to this blog.

### Update